***Problem 1-Carseats Linear Regression***

**1(a) Fit a multiple linear regression model to predict Sales using Price, Urban, and US**

In [84]:
#데이터 관리 및 조작
import pandas as pd
df=pd.read_csv('/Users/lavi/MLDL_25_2_ASSIGNMENT/ASSIGNMENT_1/Data/Carseats.csv')

#Check Files for ensure
print(df.head())


   Sales  CompPrice  Income  Advertising  Population  Price ShelveLoc  Age  \
0   9.50        138      73           11         276    120       Bad   42   
1  11.22        111      48           16         260     83      Good   65   
2  10.06        113      35           10         269     80    Medium   59   
3   7.40        117     100            4         466     97    Medium   55   
4   4.15        141      64            3         340    128       Bad   38   

   Education Urban   US  
0         17   Yes  Yes  
1         10   Yes  Yes  
2         12   Yes  Yes  
3         14   Yes  Yes  
4         13   Yes   No  


In [85]:
#변수 설정
Y=df['Sales']
X=df[['Price', 'Urban','US']]

#Check data for ensure
print(Y.head())
print(X.head())

0     9.50
1    11.22
2    10.06
3     7.40
4     4.15
Name: Sales, dtype: float64
   Price Urban   US
0    120   Yes  Yes
1     83   Yes  Yes
2     80   Yes  Yes
3     97   Yes  Yes
4    128   Yes   No


In [86]:
# Quali. var.(Urban, US) 인코딩
X_encoded = pd.get_dummies(X, drop_first=True)
print(X_encoded.head())

   Price  Urban_Yes  US_Yes
0    120       True    True
1     83       True    True
2     80       True    True
3     97       True    True
4    128       True   False


In [87]:
# 머신러닝 (Using Scikit-Learn)
from sklearn.linear_model import LinearRegression

lin_model=LinearRegression(fit_intercept=True)
lin_model.fit(X_encoded,Y)

r2=lin_model.score(X_encoded,Y)

# Output Check
print("Intercept: ",lin_model.intercept_)
print("Parameters: ", lin_model.coef_)
print("R_square: ", r2)


Intercept:  13.043468936764892
Parameters:  [-0.05445885 -0.02191615  1.2005727 ]
R_square:  0.23927539218405547


*1(a),(b) Report by pdf file.*

*(a) Which contains the fitted coefficients (including the intercept) and the training R2 of the model.*

*(b) Write the fitted model in equation form, and Provide an interpretation of each coefficient in the model.*

**1(c) Closed-form OLS and comparison**

In [88]:
import numpy as np
n=len(X_encoded)
X_closed = np.column_stack((np.ones(n), X_encoded)).astype(float)

Y_closed=Y.values.reshape(-1,1)

#check dimension X:n*(p+1), y:(n*1)
print(X_closed.shape, Y_closed.shape)

(400, 4) (400, 1)


In [89]:
beta_hat=np.linalg.inv(X_closed.T @ X_closed) @ X_closed.T @ Y_closed

print(beta_hat)

[[13.04346894]
 [-0.05445885]
 [-0.02191615]
 [ 1.2005727 ]]


In [90]:
coef_sklearn = np.concatenate(([lin_model.intercept_], lin_model.coef_))

comparison = pd.DataFrame({
    'OLS_closed_form': beta_hat.flatten(),
    'sklearn': coef_sklearn,
})
comparison['abs_diff'] = abs(comparison['OLS_closed_form'] - comparison['sklearn'])
print(comparison)

   OLS_closed_form    sklearn      abs_diff
0        13.043469  13.043469  3.907985e-14
1        -0.054459  -0.054459  1.457168e-16
2        -0.021916  -0.021916  2.942091e-15
3         1.200573   1.200573  1.998401e-15


**1(d) test H0 : βj = 0 versus H1 : βj != 0 at level α = 0.05**

In [91]:
from scipy import stats

Y_hat=X_closed @ beta_hat
#sci-kit의 LinRegression으로 도출된 coef_sklearn과 closed form의 해석적 해 모두 사용하여도 된다.
# 두 값이 거의 동일하므로... 다만 coef_sklearn은 (4,)의 차원이므로 (4,1)의 차원으로 reshape(-1,1)필요함.
epsilon=Y_closed-Y_hat

n,p_plus_1=X_closed.shape
df=n-p_plus_1

sigma_sq=(epsilon.T @ epsilon)/df
sigma=np.sqrt(sigma_sq)

var=sigma_sq*np.linalg.inv(X_closed.T @ X_closed)
se=np.sqrt(np.diag(var))

t_stat=beta_hat.flatten()/se
p_values = 2 * (1 - stats.t.cdf(np.abs(t_stat), df))


alpha = 0.05
significant = p_values < alpha

colnames = ['Intercept', 'Price', 'Urban_Yes', 'US_Yes']

t_table = pd.DataFrame({
    'Estimate': beta_hat.flatten(),
    'Std.Error': se,
    't value': t_stat,
    'Pr(>|t|)': p_values,
    'Significant (α=0.05)': significant
}, index=colnames)

print(t_table)

            Estimate  Std.Error    t value  Pr(>|t|)  Significant (α=0.05)
Intercept  13.043469   0.651012  20.035674  0.000000                  True
Price      -0.054459   0.005242 -10.389232  0.000000                  True
Urban_Yes  -0.021916   0.271650  -0.080678  0.935739                 False
US_Yes      1.200573   0.259042   4.634673  0.000005                  True


**1(e) Perform a validation set approach(7:3 random val & 5-fold cross val)**

In [92]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

random_seed = 1

X_train, X_val, y_train, y_val = train_test_split(
    X_encoded, Y, test_size=0.3, random_state=random_seed
)

#Check 7:3 sizing
print(f"Train size: {len(X_train)}, Validation size: {len(X_val)}")

Train size: 280, Validation size: 120


*7:3 Validation*

In [93]:
lm = LinearRegression(fit_intercept=True)
lm.fit(X_train, y_train)

y_pred_val = lm.predict(X_val)

r2_val = r2_score(y_val, y_pred_val)
mse_val = mean_squared_error(y_val, y_pred_val)

print(f"Validation R²: {r2_val:.4f}")
print(f"Validation MSE: {mse_val:.4f}")


Validation R²: 0.2849
Validation MSE: 5.6138


*5-fold Cross Validation*

In [94]:
lm_cv = LinearRegression(fit_intercept=True)

r2_cv_scores = cross_val_score(lm_cv, X_encoded, Y, cv=5, scoring='r2')
mse_cv_scores = -cross_val_score(lm_cv, X_encoded, Y, cv=5, scoring='neg_mean_squared_error')

r2_cv_mean = np.mean(r2_cv_scores)
mse_cv_mean = np.mean(mse_cv_scores)

print(f"Cross-validated R² (mean): {r2_cv_mean:.4f}")
print(f"Cross-validated MSE (mean): {mse_cv_mean:.4f}")

Cross-validated R² (mean): 0.2038
Cross-validated MSE (mean): 6.1903
